# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

In [ ]:
!pip install datasets sacremoses

In [3]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def collate_fn(batch):
  texts, labels = [], []
  for row in batch:
    #texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids) #단어를 tokenizer 분석처리,  Tensor 로 전환한다.
    #labels = torch.LongTensor(labels) # Label 를 Tensor로 전환한다.  0 부정평가  1 긍정평가  -> 이번 기본과제에서 skip
    labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-2])  # 과제의 필요로 2진 예측으로 부터 단어 예측으로 전환, 마지막으로 2번째 text (마지막 토큰은 end 부호니깐 넘어가서 -2로 선택합니다.)
    texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-2]))  # 마지막으로 2번째 text (입력값 x)

  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)  # pad_sequence: 매번 text의 길이를 400으로 정하고 기타 길이에 못미치는건 pad_token_id로 채웁니다.  길이에 초과되는 부분은 cutting 처리
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

Using the latest cached version of the dataset since stanfordnlp/imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /Users/yunhyeokchoi/.cache/huggingface/datasets/stanfordnlp___imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Sun Dec 22 16:16:03 2024).
Using cache found in /Users/yunhyeokchoi/.cache/torch/hub/huggingface_pytorch-transformers_main


## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [2]:
text, label = next(iter(train_loader))
print(text.shape, label.shape)
print(text[0, 1].item(), label[1].item())
print(text[0, 2].item(), label[2].item())
print(text[0, 3].item(), label[3].item())
print(text[0, 4].item(), label[4].item())
print(text[0, 5].item(), label[5].item())
print(text[0, 6].item(), label[6].item())
print(text[0, 7].item(), label[7].item())
print(text[0, 8].item(), label[8].item())
print(text[0, 9].item(), label[9].item())
print(text[0, 10].item(), label[10].item())

torch.Size([64, 400]) torch.Size([64])
12880 0
11374 0
1005 0
1055 1
2446 1
5781 0
12312 0
4247 1
1999 0
3280 1


In [ ]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model) # selfAttention 은 이 가중치들을 학습하는 과정?
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)
    result = self.dense(result)

    return result

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [ ]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff 

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential( # 토큰 백터를 개별적으로 처리후 빈선형 변환을 통해 풍부한 표현력 제공
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [ ]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [ ]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, 1)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32)

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [ ]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [ ]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 228.05881744623184
=========> Train acc: 0.770 | Test acc: 0.745
Epoch   1 | Train Loss: 181.79184779524803
=========> Train acc: 0.818 | Test acc: 0.776
Epoch   2 | Train Loss: 161.0439212024212
=========> Train acc: 0.847 | Test acc: 0.786
Epoch   3 | Train Loss: 144.77784506976604
=========> Train acc: 0.866 | Test acc: 0.798
Epoch   4 | Train Loss: 127.65684458613396
=========> Train acc: 0.892 | Test acc: 0.800
Epoch   5 | Train Loss: 113.74214302748442
=========> Train acc: 0.907 | Test acc: 0.804
Epoch   6 | Train Loss: 101.79052671045065
=========> Train acc: 0.916 | Test acc: 0.799
Epoch   7 | Train Loss: 90.75403520092368
=========> Train acc: 0.937 | Test acc: 0.809
Epoch   8 | Train Loss: 77.53205958008766
=========> Train acc: 0.945 | Test acc: 0.808
Epoch   9 | Train Loss: 68.24311332032084
=========> Train acc: 0.957 | Test acc: 0.804
Epoch  10 | Train Loss: 58.22225522249937
=========> Train acc: 0.964 | Test acc: 0.801
Epoch  11 | Train Loss: 51

학습이 안정적으로 진행되며 RNN보다 빨리 수렴하는 것을 확인할 수 있습니다.
하지만 test 정확도가 RNN보다 낮은 것을 보았을 때, overfitting에 취약하다는 것을 알 수 있습니다.

In [ ]:
import torch
from transformers import BertTokenizerFast
from datasets import load_dataset
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

ds = load_dataset("stanfordnlp/imdb") 
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


# 마지막 단어 제외하고, 해당 단어를 라벨에 넣는 collate 함수
def collate_fn(batch):
  max_len = 258 # 한 문장의 최대 토큰 갯수.
  texts, labels = [], []
  for row in batch:
    labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-2])
    texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-2])) # 2개를 자르기 때문에 최대 개수가 398 로 줄어든다.

  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
  labels = torch.LongTensor(labels)

  return texts, labels

max_word_len = 256

train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)


from torch import nn
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model) # selfAttention 은 이 가중치들을 학습하는 과정?
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)
    result = self.dense(result)

    return result
  
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff 

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential( # 토큰 백터를 개별적으로 처리후 빈선형 변환을 통해 풍부한 표현력 제공
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x
  
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


# 결과 출력을 위한 모니터
class AccuracyMonitor:
    def __init__(self, models, dataloaders, labels, title="Model Accuracies", accuracy_fn=None):
        """
        models: 모델 리스트
        dataloaders: 데이터로더 리스트
        labels: 모델 및 데이터로더에 대한 레이블 리스트
        title: 그래프 제목
        accuracy_fn: 사용자 정의 정확도 함수 (default_accuracy를 기본값으로 사용)
        """
        if not (len(models) == len(dataloaders) == len(labels)):
            raise ValueError("models, dataloaders, labels는 모두 같은 길이를 가져야 합니다.")

        self.models = models
        self.dataloaders = dataloaders
        self.labels = labels
        self.title = title
        self.acc_lists = [[] for _ in labels]
        self.accuracy_fn = accuracy_fn if accuracy_fn else self.default_accuracy

    def default_accuracy(self, model, dataloader, **kwargs):
        """
        기본 정확도 계산 함수. **kwargs를 통해 추가 옵션 지원.
        """
        cnt = 0
        acc = 0
        model.eval()
        with torch.no_grad():
            for data in dataloader:
                inputs, labels = data
                preds = model(inputs)
                preds = torch.argmax(preds, dim=-1)
                cnt += labels.shape[0]
                acc += (labels == preds).sum().item()
        model.train()
        return acc / cnt if cnt > 0 else 0.0

    def update_accuracies(self, verbose=False, **kwargs):
        """
        models와 dataloaders를 평가하고 acc_lists에 기록.
        verbose: True인 경우 정확도 업데이트 로그 출력.
        """
        for i, (model, dataloader) in enumerate(zip(self.models, self.dataloaders)):
            acc = self.accuracy_fn(model, dataloader, **kwargs)
            self.acc_lists[i].append(acc)
            if verbose:
                print(f"Updated Accuracy ({self.labels[i]}): {acc:.4f}")

    def plot(self, epoch=None, save_path=None):
        """
        정확도 그래프를 출력하고 저장하는 기능.
        epoch: 그래프 제목에 표시할 에폭 정보
        save_path: 그래프를 저장할 경로 (None이면 저장하지 않음)
        """
        if not self.acc_lists or len(self.acc_lists[0]) == 0:
            print("No accuracies to plot")
            return

        x = np.arange(len(self.acc_lists[0]))  # 에폭 수만큼 x축 생성

        plt.figure(figsize=(10, 6))  # 그래프 크기 조정
        for acc_list, label in zip(self.acc_lists, self.labels):
            plt.plot(x, acc_list, label=label, marker='o')

        title = f"{self.title} at Epoch {epoch}" if epoch is not None else self.title
        plt.title(title)
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.grid(True)
        plt.legend()
        plt.ylim(0.0, 1.0)

        if save_path:
            plt.savefig(save_path)
            print(f"Plot saved to {save_path}")
        else:
            plt.show()

device = torch.device("mps") if torch.has_mps else torch.device("cpu")


from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt

class LastWordPrediction(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_word_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id).unsqueeze(1)
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = self.classification(x)
    x = x[:, -1, :]

    return x


model = LastWordPrediction(len(tokenizer), 16, 2, 32)
lr = 0.001
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)


model = model.to(device)
def custom_accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = torch.argmax(preds, dim=-1) # 가장 높은 확률을 가진 클래스를 선택

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


n_epochs = 50

trans_monitor = AccuracyMonitor(
    models=[model, model],
    dataloaders=[train_loader, test_loader],
    labels=["Train", "Test"],
    accuracy_fn=custom_accuracy,
)

for epoch in range(n_epochs):
    total_loss = 0.
    model.train()
    for data in train_loader:
        model.zero_grad()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        labels = labels.long()

        preds = model(inputs)
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(total_loss)
    print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")
    
  
    with torch.no_grad():
        model.eval()
        trans_monitor.update_accuracies()
    if (epoch + 1) % 3 == 0:  # 10 epoch마다 그래프 업데이트
        trans_monitor.plot()